<a href="https://www.kaggle.com/code/jithendrapuppala/nfl-2026-big-data-bowl-baseline-prediction?scriptVersionId=270915839" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-big-data-bowl-2026-prediction/test_input.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/test.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/nfl_inference_server.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/nfl_gateway.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/__init__.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/templates.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/base_gateway.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/relay.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/__init__.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/generated/kaggle_evaluation_pb2.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/generated/kaggle_evaluati

In [2]:
ROOT_DIR = '/kaggle/input/nfl-big-data-bowl-2026-prediction/'
test_inp = pd.read_csv(os.path.join(ROOT_DIR, 'test_input.csv'))
test = pd.read_csv(os.path.join(ROOT_DIR, 'test.csv'))

test_inp.shape, test.shape

((49753, 23), (5837, 5))

In [3]:
test.head()

,id,game_id,play_id,nfl_id,frame_id
0,12350,2024120805,74,54586,1
1,12351,2024120805,74,54586,2
2,12352,2024120805,74,54586,3
3,12353,2024120805,74,54586,4
4,12354,2024120805,74,54586,5


In [4]:
TRAIN_DIR = os.path.join(ROOT_DIR, 'train')
input_files = []
output_files = []

def is_filetype(filename, filetype='input'):
    type_len = len(filetype)
    if len(filename)>=type_len:
        if filename[:type_len]==filetype:
            return True
    return False
    
for filename in os.listdir(TRAIN_DIR):
    if is_filetype(filename, 'input'):
        input_files.append(filename)
    elif is_filetype(filename, 'output'):
        output_files.append(filename)

input_files.sort()
output_files.sort()

print(input_files)
print(output_files)

['input_2023_w01.csv', 'input_2023_w02.csv', 'input_2023_w03.csv', 'input_2023_w04.csv', 'input_2023_w05.csv', 'input_2023_w06.csv', 'input_2023_w07.csv', 'input_2023_w08.csv', 'input_2023_w09.csv', 'input_2023_w10.csv', 'input_2023_w11.csv', 'input_2023_w12.csv', 'input_2023_w13.csv', 'input_2023_w14.csv', 'input_2023_w15.csv', 'input_2023_w16.csv', 'input_2023_w17.csv', 'input_2023_w18.csv']
['output_2023_w01.csv', 'output_2023_w02.csv', 'output_2023_w03.csv', 'output_2023_w04.csv', 'output_2023_w05.csv', 'output_2023_w06.csv', 'output_2023_w07.csv', 'output_2023_w08.csv', 'output_2023_w09.csv', 'output_2023_w10.csv', 'output_2023_w11.csv', 'output_2023_w12.csv', 'output_2023_w13.csv', 'output_2023_w14.csv', 'output_2023_w15.csv', 'output_2023_w16.csv', 'output_2023_w17.csv', 'output_2023_w18.csv']


In [5]:
inp_file = input_files[0]
out_file = output_files[0]
inp = pd.read_csv(os.path.join(TRAIN_DIR, inp_file))
out = pd.read_csv(os.path.join(TRAIN_DIR, out_file))

display(inp.head())
display(out.head())
print(inp.shape, out.shape)

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
0,2023090700,101,False,54527,1,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.09,0.39,322.40,238.24,21,63.259998,-0.22
1,2023090700,101,False,54527,2,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.04,0.61,200.89,236.05,21,63.259998,-0.22
2,2023090700,101,False,54527,3,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.93,0.12,0.73,147.55,240.60,21,63.259998,-0.22
3,2023090700,101,False,54527,4,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.35,36.92,0.23,0.81,131.40,244.25,21,63.259998,-0.22
4,2023090700,101,False,54527,5,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.37,36.90,0.35,0.82,123.26,244.25,21,63.259998,-0.22


,game_id,play_id,nfl_id,frame_id,x,y
0,2023090700,101,46137,1,56.22,17.28
1,2023090700,101,46137,2,56.63,16.88
2,2023090700,101,46137,3,57.06,16.46
3,2023090700,101,46137,4,57.48,16.02
4,2023090700,101,46137,5,57.91,15.56


(285714, 23) (32088, 6)


In [6]:
inp_out_map_cols = ['game_id', 'play_id', 'nfl_id', 'frame_id']

def create_id(df, id_cols):
    N = len(id_cols)
    df['id'] = ''
    for i in range(N):
        df['id'] += df[id_cols[i]].astype(str)
        if i+1!=N:
            df['id'] += '_'
    df = df.drop(columns = id_cols).copy()
    return df

inp_f = create_id(inp, inp_out_map_cols)
out_f = create_id(out, inp_out_map_cols)

display(inp_f.head())
display(out_f.head())

,player_to_predict,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,player_position,player_side,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y,id
0,False,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.33,36.94,0.09,0.39,322.40,238.24,21,63.259998,-0.22,2023090700_101_54527_1
1,False,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.33,36.94,0.04,0.61,200.89,236.05,21,63.259998,-0.22,2023090700_101_54527_2
2,False,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.33,36.93,0.12,0.73,147.55,240.60,21,63.259998,-0.22,2023090700_101_54527_3
3,False,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.35,36.92,0.23,0.81,131.40,244.25,21,63.259998,-0.22,2023090700_101_54527_4
4,False,right,42,Bryan Cook,6-1,210,1999-09-07,FS,Defense,Defensive Coverage,52.37,36.90,0.35,0.82,123.26,244.25,21,63.259998,-0.22,2023090700_101_54527_5


,x,y,id
0,56.22,17.28,2023090700_101_46137_1
1,56.63,16.88,2023090700_101_46137_2
2,57.06,16.46,2023090700_101_46137_3
3,57.48,16.02,2023090700_101_46137_4
4,57.91,15.56,2023090700_101_46137_5


In [7]:
df = pd.merge(out, inp, on=['id'], how='inner')
df.shape

(31446, 30)

In [8]:
inp.shape, out.shape

((285714, 24), (32088, 7))

In [9]:
df.head()

,game_id_x,play_id_x,nfl_id_x,frame_id_x,x_x,y_x,id,game_id_y,play_id_y,player_to_predict,...,player_role,x_y,y_y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
0,2023090700,101,46137,1,56.22,17.28,2023090700_101_46137_1,2023090700,101,True,...,Defensive Coverage,51.32,20.69,0.31,0.49,79.43,267.68,21,63.259998,-0.22
1,2023090700,101,46137,2,56.63,16.88,2023090700_101_46137_2,2023090700,101,True,...,Defensive Coverage,51.35,20.66,0.36,0.74,118.07,268.66,21,63.259998,-0.22
2,2023090700,101,46137,3,57.06,16.46,2023090700_101_46137_3,2023090700,101,True,...,Defensive Coverage,51.39,20.63,0.44,0.76,130.89,269.78,21,63.259998,-0.22
3,2023090700,101,46137,4,57.48,16.02,2023090700_101_46137_4,2023090700,101,True,...,Defensive Coverage,51.43,20.61,0.48,0.62,134.50,269.78,21,63.259998,-0.22
4,2023090700,101,46137,5,57.91,15.56,2023090700_101_46137_5,2023090700,101,True,...,Defensive Coverage,51.48,20.58,0.54,0.44,129.79,269.06,21,63.259998,-0.22
